In [2]:
import numpy as np
import pandas as pd
from typing import Union

In [3]:
train = pd.read_csv("../data/train_preprocessed.csv")
train_x = train.drop(["target"], axis=1)
train_y = train["target"]

test_x = pd.read_csv("../data/test_preprocessed.csv")
test_x = test_x.dropna(how="any")

In [4]:
# モデルのスタッキング
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold

from models import Model2Linear, ModleGBDT

In [ ]:
def predict_cv(model:Union[ModleGBDT, Model2Linear], train_x:pd.DataFrame, train_y:pd.DataFrame, test_x:pd.DataFrame):
    """
    学習データに対する目的変数を知らない学習、予測を行い、テストデータに対する予測値を返す関数
    """
    preds = []
    preds_test = []
    va_idxes = []

    kf = KFold(n_splits=4, shuffle=True, random_state=71)

    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

        model.fit(tr_x, tr_y, va_x, va_y)

        pred = model.predict(va_x)
        preds.append(pred)

        pred_test = model.predict(test_x)
        preds_test.append(pred_test)

        va_idxes.append(va_idx)
    
    # validationに対する予測値を連結、元の順序に並べなおす
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)

    order = np.argsort(va_idxes)
    pred_train = preds[order]

    pred_test = np.mean(pred_test, axis=0)

    return pred_train, pred_test